In [31]:
import pandas as pd
df=pd.read_csv("crx.data",header=None,na_values="?")
#print(df.isna().sum())
#print(df.head())
from sklearn.impute import SimpleImputer
df_im=pd.DataFrame(SimpleImputer(strategy="most_frequent").fit_transform(df))
#print(df_im.isna().sum())
from sklearn.preprocessing import LabelEncoder
for i in [0,3,4,5,6,8,9,11,12,15]:
    df_im[i]=LabelEncoder().fit_transform(df_im[i])
print(df_im.head())

   0      1      2   3   4   5   6     7   8   9  10  11  12     13   14  15
0   1  30.83    0.0   1   0  12   7  1.25   1   1  1   0   0  202.0    0   0
1   0  58.67   4.46   1   0  10   3  3.04   1   1  6   0   0   43.0  560   0
2   0   24.5    0.5   1   0  10   3   1.5   1   0  0   0   0  280.0  824   0
3   1  27.83   1.54   1   0  12   7  3.75   1   1  5   1   0  100.0    3   0
4   1  20.17  5.625   1   0  12   7  1.71   1   0  0   0   2  120.0    0   0


In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [33]:
pipe=Pipeline([('sc',StandardScaler()),
               ('pca',PCA()),
               ('svmr',SVC(kernel="rbf"))])
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df_im.drop(columns=[15]),
                                          df_im[15],
                                          test_size=0.3)
pipe.fit(xtrain,ytrain)

Pipeline(steps=[('sc', StandardScaler()), ('pca', PCA()), ('svmr', SVC())])

In [19]:
print(pipe.score(xtrain,ytrain))
print(pipe.score(xtest,ytest))

0.8923395445134575
0.8743961352657005


In [34]:
''''''
data=df_im.drop(columns=[15])
target=df_im[15]
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=10)
for trainindex,testindex in skf.split(data,target):
    xtrain,xtest=data.iloc[trainindex],data.iloc[testindex]
    ytrain,ytest=target[trainindex],target[testindex]
    pipe.fit(xtrain,ytrain)
    print(pipe.score(xtest,ytest))


0.43478260869565216
0.8115942028985508
0.9855072463768116
0.9420289855072463
0.9420289855072463
0.927536231884058
0.8695652173913043
0.6231884057971014
1.0
0.855072463768116


In [40]:
from sklearn.model_selection import KFold
data=df_im.drop(columns=[15])
target=df_im[15]
kf=KFold(n_splits=3)
for train,test in kf.split(data,target):
    xtrain,xtest=data.iloc[train],data.iloc[test]
    ytrain,ytest=target.iloc[train], target.iloc[test]
    pipe.fit(xtrain,ytrain)
    print(pipe.score(xtest,ytest))
    

0.7869565217391304
0.9347826086956522
0.8521739130434782


In [44]:
from sklearn.model_selection import StratifiedKFold
data=df_im.drop(columns=[15])
target=df_im[15]
kf=StratifiedKFold(n_splits=3)
for train,test in kf.split(data,target):
    xtrain,xtest=data.iloc[train],data.iloc[test]
    ytrain,ytest=target.iloc[train], target.iloc[test]
    pipe.fit(xtrain,ytrain)
    print(pipe.score(xtest,ytest))

0.7608695652173914
0.9260869565217391
0.8217391304347826


In [46]:
from sklearn.model_selection import cross_validate
for i in range(3,11):
  cv_results=cross_validate(pipe, data, target, cv=i)
  print("K Fold: ", i)
  print(np.average(cv_results['test_score']))
    

K Fold:  3
0.836231884057971
K Fold:  4
0.8377470090065868
K Fold:  5
0.846376811594203
K Fold:  6
0.8405797101449276
K Fold:  7
0.8420502400094237
K Fold:  8
0.8409850307404437
K Fold:  9
0.8418964076858813
K Fold:  10
0.8391304347826086


In [53]:
from sklearn import datasets
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
data=datasets.load_breast_cancer()
X=data.data
Y=data.target
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3)
model=SVC()
model.fit(xtrain,ytrain)
print(confusion_matrix(ytest,model.predict(xtest)))
print(classification_report(ytest,model.predict(xtest)))

[[55 13]
 [ 4 99]]
              precision    recall  f1-score   support

           0       0.93      0.81      0.87        68
           1       0.88      0.96      0.92       103

    accuracy                           0.90       171
   macro avg       0.91      0.88      0.89       171
weighted avg       0.90      0.90      0.90       171



In [54]:
from sklearn.model_selection import GridSearchCV

In [56]:
param_grid = {'C': [0.1, 1, 10, 100],  
              'kernel': ['linear','poly','rbf']}  
   
grid = GridSearchCV(SVC(), param_grid) 
   
# fitting the model for grid search 
grid.fit(xtrain, ytrain) 
 
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(xtest) 
   
# print classification report 
print(classification_report(ytest, grid_predictions)) 

{'C': 10, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        68
           1       0.94      1.00      0.97       103

    accuracy                           0.96       171
   macro avg       0.97      0.96      0.96       171
weighted avg       0.97      0.96      0.96       171

